In [1]:
import pyspark.sql.functions as f 

In [2]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("CleaningData")\
    .config('spark.driver.extraClassPath','/usr/lib/jvm/java-17-openjdk-amd64/lib/postgresql-42.6.0.jar')\
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
23/10/25 09:18:23 WARN Utils: Your hostname, LAPTOP-GRLQJBNM resolves to a loopback address: 127.0.1.1; using 172.27.87.196 instead (on interface eth0)
23/10/25 09:18:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/25 09:18:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from dotenv import load_dotenv
import os

load_dotenv()

user = os.getenv("user")
password = os.getenv("password")

In [4]:
jdbc_url = "jdbc:postgresql://localhost:5432/Apps_Database"

connection_properties = {
    "user": user,
    "password": password,
    "driver": "org.postgresql.Driver"
}

uncleaned_data = spark.read.jdbc(url=jdbc_url, table="apps_raw_data_table", properties=connection_properties)


In [5]:
uncleaned_data.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



In [6]:
uncleaned_data.show()

+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------------+------------------+------------+
|                 App|      Category|Rating|Reviews|Size|   Installs|Type|Price|Content Rating|              Genres|      Last Updated|       Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------------+------------------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M|    10,000+|Free|    0|      Everyone|        Art & Design|   January 7, 2018|             1.0.0|4.0.3 and up|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967| 14M|   500,000+|Free|    0|      Everyone|Art & Design;Pret...|  January 15, 2018|             2.0.0|4.0.3 and up|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510|8.7M| 5,000,000+|Free|    0|      Everyone|        Art & Design|    August 1, 2018|             1.2.4|4.0.3 

### Checking for the null values

In [7]:
# Checking for the count of  null values present in all the columns

uncleaned_data.select([f.count(f.when(f.isnan(c) | f.col(c).isNull(), c)).alias(c) for c in uncleaned_data.columns]).show()

+---+--------+------+-------+----+--------+----+-----+--------------+------+------------+-----------+-----------+
|App|Category|Rating|Reviews|Size|Installs|Type|Price|Content Rating|Genres|Last Updated|Current Ver|Android Ver|
+---+--------+------+-------+----+--------+----+-----+--------------+------+------------+-----------+-----------+
|  0|       0|  1474|      0|   0|       0|   1|    0|             1|     0|           0|          8|          3|
+---+--------+------+-------+----+--------+----+-----+--------------+------+------------+-----------+-----------+



In [13]:
uncleaned_data = uncleaned_data.withColumn("Rating", f.when(f.col("Rating") == "NaN", "0.0").otherwise(f.col("Rating")))

In [14]:
uncleaned_data.show(100)

+--------------------+-----------------+------+-------+------------------+-----------+----+-----+--------------+--------------------+------------------+------------------+------------------+
|                 App|         Category|Rating|Reviews|              Size|   Installs|Type|Price|Content Rating|              Genres|      Last Updated|       Current Ver|       Android Ver|
+--------------------+-----------------+------+-------+------------------+-----------+----+-----+--------------+--------------------+------------------+------------------+------------------+
|Photo Editor & Ca...|   ART_AND_DESIGN|   4.1|    159|               19M|    10,000+|Free|    0|      Everyone|        Art & Design|   January 7, 2018|             1.0.0|      4.0.3 and up|
| Coloring book moana|   ART_AND_DESIGN|   3.9|    967|               14M|   500,000+|Free|    0|      Everyone|Art & Design;Pret...|  January 15, 2018|             2.0.0|      4.0.3 and up|
|U Launcher Lite –...|   ART_AND_DESIGN|   4.

In [18]:
uncleaned_data.select([f.count(f.when(f.isnan(c) | f.col(c).isNull(), c)).alias(c) for c in uncleaned_data.columns]).show()

+---+--------+------+-------+----+--------+----+-----+--------------+------+------------+-----------+-----------+
|App|Category|Rating|Reviews|Size|Installs|Type|Price|Content Rating|Genres|Last Updated|Current Ver|Android Ver|
+---+--------+------+-------+----+--------+----+-----+--------------+------+------------+-----------+-----------+
|  0|       0|     0|      0|   0|       0|   1|    0|             1|     0|           0|          8|          3|
+---+--------+------+-------+----+--------+----+-----+--------------+------+------------+-----------+-----------+



In [16]:
# Removing + from installs

uncleaned_data = uncleaned_data.withColumn("Installs", f.regexp_replace(f.col("Installs"), "\\+", ""))
uncleaned_data.show()

+--------------------+--------------+------+-------+----+----------+----+-----+--------------+--------------------+------------------+------------------+------------+
|                 App|      Category|Rating|Reviews|Size|  Installs|Type|Price|Content Rating|              Genres|      Last Updated|       Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+----------+----+-----+--------------+--------------------+------------------+------------------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M|    10,000|Free|    0|      Everyone|        Art & Design|   January 7, 2018|             1.0.0|4.0.3 and up|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967| 14M|   500,000|Free|    0|      Everyone|Art & Design;Pret...|  January 15, 2018|             2.0.0|4.0.3 and up|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510|8.7M| 5,000,000|Free|    0|      Everyone|        Art & Design|    August 1, 2018|             1.2.4|4.0.3 and up

In [17]:
uncleaned_data = uncleaned_data.na.drop(subset=["Type"])

In [19]:
uncleaned_data.select([f.count(f.when(f.isnan(c) | f.col(c).isNull(), c)).alias(c) for c in uncleaned_data.columns]).show()

+---+--------+------+-------+----+--------+----+-----+--------------+------+------------+-----------+-----------+
|App|Category|Rating|Reviews|Size|Installs|Type|Price|Content Rating|Genres|Last Updated|Current Ver|Android Ver|
+---+--------+------+-------+----+--------+----+-----+--------------+------+------------+-----------+-----------+
|  0|       0|     0|      0|   0|       0|   1|    0|             1|     0|           0|          8|          3|
+---+--------+------+-------+----+--------+----+-----+--------------+------+------------+-----------+-----------+



In [24]:
testing_data = uncleaned_data.na.drop(how="any")
check = testing_data.select([f.count(f.when(f.isnan(c) | f.col(c).isNull(), c)).alias(c) for c in testing_data.columns])

In [25]:
check.show()

+---+--------+------+-------+----+--------+----+-----+--------------+------+------------+-----------+-----------+
|App|Category|Rating|Reviews|Size|Installs|Type|Price|Content Rating|Genres|Last Updated|Current Ver|Android Ver|
+---+--------+------+-------+----+--------+----+-----+--------------+------+------------+-----------+-----------+
|  0|       0|     0|      0|   0|       0|   1|    0|             0|     0|           0|          7|          2|
+---+--------+------+-------+----+--------+----+-----+--------------+------+------------+-----------+-----------+



In [29]:
columns_to_filter = []

# Iterate over the column names and check if the count is greater than 1
for column in check.columns:
    count_value = check.select(column).first()[0]
    if count_value > 1:
        columns_to_filter.append(column)

# Filter and print rows where any of the specified columns have counts greater than 1
filtered_data = testing_data.filter(~(
    (f.col(columns_to_filter[0]).isNull() | f.isnan(columns_to_filter[0])) |
    (f.col(columns_to_filter[1]).isNull() | f.isnan(columns_to_filter[1])) ))
    # Add more columns as needed)
filtered_data.show()

+--------------------+--------------+------+-------+----+----------+----+-----+--------------+--------------------+------------------+------------------+------------+
|                 App|      Category|Rating|Reviews|Size|  Installs|Type|Price|Content Rating|              Genres|      Last Updated|       Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+----------+----+-----+--------------+--------------------+------------------+------------------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M|    10,000|Free|    0|      Everyone|        Art & Design|   January 7, 2018|             1.0.0|4.0.3 and up|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967| 14M|   500,000|Free|    0|      Everyone|Art & Design;Pret...|  January 15, 2018|             2.0.0|4.0.3 and up|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510|8.7M| 5,000,000|Free|    0|      Everyone|        Art & Design|    August 1, 2018|             1.2.4|4.0.3 and up

In [30]:
filtered_data.select([f.count(f.when(f.isnan(c) | f.col(c).isNull(), c)).alias(c) for c in filtered_data.columns]).show()

+---+--------+------+-------+----+--------+----+-----+--------------+------+------------+-----------+-----------+
|App|Category|Rating|Reviews|Size|Installs|Type|Price|Content Rating|Genres|Last Updated|Current Ver|Android Ver|
+---+--------+------+-------+----+--------+----+-----+--------------+------+------------+-----------+-----------+
|  0|       0|     0|      0|   0|       0|   1|    0|             0|     0|           0|          0|          0|
+---+--------+------+-------+----+--------+----+-----+--------------+------+------------+-----------+-----------+



In [31]:
null_removed_data = filtered_data

In [33]:
null_removed_data.show(100)

+--------------------+-----------------+------+-------+------------------+----------+----+-----+--------------+--------------------+------------------+------------------+------------------+
|                 App|         Category|Rating|Reviews|              Size|  Installs|Type|Price|Content Rating|              Genres|      Last Updated|       Current Ver|       Android Ver|
+--------------------+-----------------+------+-------+------------------+----------+----+-----+--------------+--------------------+------------------+------------------+------------------+
|Photo Editor & Ca...|   ART_AND_DESIGN|   4.1|    159|               19M|    10,000|Free|    0|      Everyone|        Art & Design|   January 7, 2018|             1.0.0|      4.0.3 and up|
| Coloring book moana|   ART_AND_DESIGN|   3.9|    967|               14M|   500,000|Free|    0|      Everyone|Art & Design;Pret...|  January 15, 2018|             2.0.0|      4.0.3 and up|
|U Launcher Lite –...|   ART_AND_DESIGN|   4.7|  8

In [34]:
varies_changed = null_removed_data.withColumn("Size", f.when(f.col("Size") == "Varies with device", None).otherwise(f.col("Size")))

In [35]:
varies_changed.show(100)

+--------------------+-----------------+------+-------+----+----------+----+-----+--------------+--------------------+------------------+------------------+------------------+
|                 App|         Category|Rating|Reviews|Size|  Installs|Type|Price|Content Rating|              Genres|      Last Updated|       Current Ver|       Android Ver|
+--------------------+-----------------+------+-------+----+----------+----+-----+--------------+--------------------+------------------+------------------+------------------+
|Photo Editor & Ca...|   ART_AND_DESIGN|   4.1|    159| 19M|    10,000|Free|    0|      Everyone|        Art & Design|   January 7, 2018|             1.0.0|      4.0.3 and up|
| Coloring book moana|   ART_AND_DESIGN|   3.9|    967| 14M|   500,000|Free|    0|      Everyone|Art & Design;Pret...|  January 15, 2018|             2.0.0|      4.0.3 and up|
|U Launcher Lite –...|   ART_AND_DESIGN|   4.7|  87510|8.7M| 5,000,000|Free|    0|      Everyone|        Art & Design|  

In [37]:
cleaning_data = varies_changed.withColumn("size_dependency", f.when(f.col("Size").isNull(), False).otherwise(True))
cleaning_data.show(100)

+--------------------+-----------------+------+-------+----+----------+----+-----+--------------+--------------------+------------------+------------------+------------------+---------------+
|                 App|         Category|Rating|Reviews|Size|  Installs|Type|Price|Content Rating|              Genres|      Last Updated|       Current Ver|       Android Ver|size_dependency|
+--------------------+-----------------+------+-------+----+----------+----+-----+--------------+--------------------+------------------+------------------+------------------+---------------+
|Photo Editor & Ca...|   ART_AND_DESIGN|   4.1|    159| 19M|    10,000|Free|    0|      Everyone|        Art & Design|   January 7, 2018|             1.0.0|      4.0.3 and up|           true|
| Coloring book moana|   ART_AND_DESIGN|   3.9|    967| 14M|   500,000|Free|    0|      Everyone|Art & Design;Pret...|  January 15, 2018|             2.0.0|      4.0.3 and up|           true|
|U Launcher Lite –...|   ART_AND_DESIGN|